- enviroment


In [ ]:
import gensim
import spacy
from scipy import spatial
import json
import numpy as np
import nltk
from gensim.models import Word2Vec, Doc2Vec
from gensim import models
import string
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from collections import defaultdict
from math import log
from textblob import TextBlob
from collections import Counter
from nltk.tag import StanfordNERTagger

%matplotlib inline

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile
import tarfile
 
import hashlib
import re
import itertools
from sklearn import metrics



- data file

In [288]:
train_set = "/Users/alfredchen/Documents/GitHub/query-system/data/training.json"
dev_set = "/Users/alfredchen/Documents/GitHub/query-system/data/devel.json"
test_set = "/Users/alfredchen/Documents/GitHub/query-system/data/testing.json"
doc = "/Users/alfredchen/Documents/GitHub/query-system/data/documents.json"

#save word embeddings
embeddings = "/Users/alfredchen/Documents/GitHub/query-system/models/mymodel-size"

# save qa log
log_file = "/Users/alfredchen/Documents/GitHub/query-system/data/log.txt"
answer_type_file = "/Users/alfredchen/Documents/GitHub/query-system/data/answer_type_label.txt"
topk_file = "/Users/alfredchen/Documents/GitHub/query-system/data/topk.json"
answer_file = "/Users/alfredchen/Documents/GitHub/query-system/data/answer.txt"
ner_jar = "/Users/alfredchen/Documents/GitHub/query-system/data/stanford-ner.jar"
ner_model = "/Users/alfredchen/Documents/GitHub/query-system/data/english.muc.7class.distsim.crf.ser.gz"

- preprocess corpus
    - remove stopwords
    - lemmatize
    - lower case
    - creat paragraph_index
    - create index2paragraph

In [308]:
    def get_stopwords(flag):
        
        words = list(nltk.corpus.stopwords.words())
        
        if flag == False:
        #some stopwords are helpful in targeting answer type, shall not be removed
            words.remove('what')
            words.remove('where')
            words.remove('when')
            words.remove('who')
            words.remove('how')
            words.remove('which')
        # add more #

        stopwords = {}
        for word in words:
            stopwords[word] = stopwords.get(word,0) + 1
        return stopwords

In [4]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma

def removePunc(string):
    # define punctuation
    punctuations = '''!()-[]{};:'"\,./?@#$%^&*_~'''
    
    # remove punctuation from the string
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    #print(no_punct)
    return no_punct


stopwords = get_stopwords(True)        # get stopwords

def preprocess_docs(corpus):
        para2index = {}
        index2para = {}
        new_corpus = []
        para_len = 0
        para_count = 0
        for _id, doc in enumerate(corpus):
            new_doc = []
            for _para,para in enumerate(doc['text']):
                para2index[para] = (_id,_para)         # para index
                index2para[(_id,_para)] = para         # doc_id, answer_para index
                new_para = []
                sents = para.split('.')
                para_count += 1
                for _sent,sent in enumerate(sents):
                    new_sent=[]
                    sent = removePunc(sent)
                    #sent = '<s> ' + sent + ' <end>'  # padding
                    words = sent.split(' ')
                    s = ''.join(sent)
                    if s == '<s> <end>':
                        continue
                    for word in words:
                        word = word.strip(',')
                        word = word.strip(',')
                        word = word.strip('.')
                        word = word.strip('?')
                        word = word.strip('\'s')
                        word = word.strip('\"')
                            #if not word.isalpha():
                                #continue
                        new_word = word.lower()
                        new_word = lemmatize(new_word)
                        if stopwords.get(new_word):
                            continue
                        if new_word == '':
                            continue
                        para_len += 1
                        new_sent.append(new_word)                
                    new_para.append(new_sent)
                new_doc.append(new_para)
            new_corpus.append(new_doc)
        average_para_len = (para_len/para_count)
        print(average_para_len)
        return new_corpus, para2index, index2para

In [5]:
docs = json.load(open(doc))
corpus,para2index,index2para = preprocess_docs(docs)

71.1837880794702


- get word embeddings based on pre processed corpus

In [6]:
    def w2v(corpora,size,iter):
        docs = corpora
        sentences = []
        for doc in docs:
            for sents in doc:
                new_para = []
                for sent in sents:
                    new_para += sent
                sentences.append(new_para)
        model = Word2Vec(sentences, size=size, window=5,iter=iter,workers=4)
        model.save(embeddings+str(size)+'-iter'+str(iter))
        return model

In [284]:
#model = w2v(corpus,160,150)   # train the model

#load pre-trained model
model = gensim.models.Word2Vec.load(embeddings+str(160)+'-iter'+str(150))

In [8]:
def term_freqs(corpus):
    tfs = defaultdict(dict)
    tfs_forward = defaultdict(dict)
    doc_id = 0
    for doc in corpus:
        for para in doc:
            for sent in para:
                for term in sent:
                    tfs[term][doc_id] = tfs[term].get(doc_id, 0) + 1 
                    tfs_forward[doc_id][term] = tfs[doc_id].get(term, 0) + 1
        doc_id += 1
    return tfs,doc_id+1,tfs_forward


tfs,total_docment,tfs_forward = term_freqs(corpus)

def get_tfidf(tfs, total_docment,tfs_forward):
    document_length = {}
    for doc_id,doc_list in tfs_forward.items():
        length = 0
        for term, freq in doc_list.items():
            length += freq ** 2
        length = length **0.5
        document_length[doc_id] = length
    tfidf = defaultdict(dict)
    for term, doc_list in tfs.items():
        df = len(doc_list)
        for doc_id, freq in doc_list.items(): 
            tfidf[term][doc_id] = (float(tfs[term][doc_id]) * log(total_docment / df)) / document_length.get(doc_id)
    return tfidf

tfidf = get_tfidf(tfs, total_docment,tfs_forward)






- transform sentence to feature vector (sentence embeddings)

In [40]:
    index2word_set = set(model.wv.index2word)
    
    def get_weight(word,doc_id):
        try:
            weight = tfidf[word][doc_id]
        except KeyError:
            weight = 0.0001
        return weight 
    
    def sent2vec(sentence, model, size, index2word_set, doc_id):
        """transform word embeddings to sentence vector
        param: sentence: sentence that want to be transformed
        param: model: pre-trained word embeddings
        param: size: feature vector dimension
        param: index2word_set
        return: transformed sentence vector
        """        
        try:
            words = sentence.split()  
        except:
            words = sentence
        
        try:
            words = [removePunc(x) for x in words]
        except:
            pass
        feature_vec = np.zeros((size,), dtype='float32')
        n_words = 0
        for word in words:
            word = word.lower()
            word = lemmatize(word)
            if stopwords.get(word):
                continue
            if word in index2word_set:
                n_words += 1
                if doc_id != 'disable':
                    w = get_weight(word,doc_id)
                else:
                    w = 1
                feature_vec = np.add(feature_vec, model.wv[word]*w)
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

- play with sentence vector

In [10]:
# calculate similarity of two sentence

s1_afv = sent2vec('A kilogram could be definined as having a Planck constant of what value?', model, 160, index2word_set,'disable')
s2_afv = sent2vec('Possible new definitions include "the mass of a body at rest whose equivalent energy equals the energy of photons whose frequencies sum to 7050135639273999999♠135639274×1042 Hz", or simply "the kilogram is defined so that the Planck constant equals 6966662606895999999♠6.62606896×10−34 J⋅s".', model, 160, index2word_set,'disable')
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)
model.wv.most_similar_cosmul(positive=['male', 'queen'], negative=['female'])


0.6428887844085693


[('king', 0.7751615643501282),
 ('balmoral', 0.748808741569519),
 ('duke', 0.7470282912254333),
 ('fief', 0.7448990941047668),
 ('royal', 0.7361407279968262),
 ('lambert', 0.7349075078964233),
 ('controversially', 0.7227569818496704),
 ('prince', 0.7206754088401794),
 ('frederick', 0.7180166244506836),
 ('elizabeth', 0.7173427939414978)]

- get similarity ranking of query to paragraphs in a documents

In [11]:
def sim_sent(query, corpus):
    sim2ipara = {}
    index2sim = {}
    for _id,doc in enumerate(corpus):
        q = sent2vec(query, model, 160, index2word_set,_id)
        for _para,para in enumerate(doc):
            sentence = []
            for sent in para:
                sentence += sent
            sentence = sent2vec(sentence, model, 160, index2word_set,'disable')
            sim = 1 - spatial.distance.cosine(q, sentence)
            index2sim[_id, _para] = sim
            sim2ipara[sim] = (_id,_para)
    return index2sim, sim2ipara
    

#index2sim, sim2ipara = sim_sent("what does the Planck constant refer to?",corpus)



In [12]:
def sentence_similarity(query, answer):
    sim = 1 - spatial.distance.cosine(query, answer)
    return sim
    

In [458]:
from nltk.tokenize import word_tokenize

def getTopN(N,query,docid):
    if docid == 'disable':
        top_document_id = Counter()
        for token in word_tokenize(query):
            token = token.lower()
            token = lemmatize(token)
            if not stopwords.get(token):
                term_tfidf = tfidf[token]
                for docid, weight in term_tfidf.items():
                     top_document_id[docid] += weight
        top_document_id = top_document_id.most_common(N)
        top_document = []
        for document_id,weight in top_document_id:
            top_document.append(document_id)
    else:
        top_document = [docid]
    sim2index = {}
    q = sent2vec(query, model, 160, index2word_set,'disable')
    
    for _id in top_document:
        #q = sent2vec(query, model, 160, index2word_set,_id)
        paras = [para for (doc,para) in index2para.keys() if doc == _id]
        for _para in paras:
            answer = index2para.get((_id,_para))
            answer = sent2vec(answer, model, 160, index2word_set,'disable')
            sim = sentence_similarity(q, answer)
            sim2index[sim] = (_id,_para)
    
    similarity_ranks = sorted(sim2index.keys(),reverse=True)   #sims
    rank = []
    for i in range(N):
        try:
            rank.append(sim2index.get(similarity_ranks[i]))
        except IndexError as e:
            print(top_document_id)
    logs_file = open(log_file,"a")
    logs_file.write(str(rank)+'\n')
    logs_file.close()
    return rank
    
        

index = getTopN(10,
                "A kilogram could be definined as having a Planck constant of what value?",'disable')    
print(index)

    

[(0, 5), (0, 1), (0, 16), (0, 13), (0, 23), (0, 11), (0, 17), (0, 2), (0, 10), (0, 0)]


- get top n answer paragraph given quey

In [459]:
"""
def getTopN(N,query):
    index2sim, sim2ipara = sim_sent(query,corpus)
    
    similarity_ranks = sorted(index2sim.values(),reverse=True)   #sims
    rank = []
    for i in range(N):
        rank.append(sim2ipara.get(similarity_ranks[i])) 
        
    return rank
    

getTopN(10,"Who developed the theory of inheritance known as pangenesis?")
"""
pass

- test hit accuracy: return top 10 paragraphs, is given answer included?

In [460]:
def gold_standard(dataset):
    file = json.load(open(dataset))
    gold2index={}
    for line in file:
        _id = line['docid']
        try:
            _para = line['answer_paragraph']
        except KeyError:
            _para = line['id']
        gold2index[line['question']]=(_id,_para)
    return gold2index

def my_qa(n,dataset):
    file = json.load(open(dataset))
    qa2index={}
    for i in range(n):
        save = open(log_file,"a")
        line = file[i]
        query = line['question']
        possible = getTopN(10,query,line[docid])
        qa2index[line['question']] = possible
        record = line['question']+':'+str(possible)+'\n'
        save.write(record)
        save.close()
    return qa2index
    
    

In [461]:

def acc(gold_standard,qas):
    query = qas.keys()
    total = 0
    correct = 0
    for i,q in enumerate(query):
        gold = gold_standard.get(q)
        if gold in qas.get(q):
            correct += 1
        total += 1
    acc = correct/total
    return acc



In [462]:
"""
#test on top 100 trainset
gold_index = gold_standard(train_set)
qa_index = my_qa(100,train_set)
accu = acc(gold_index,qa_index)
print(accu)
"""
pass



#test on entire devtset
gold_index = gold_standard(dev_set)
qa_index = my_qa(100,dev_set)
acc = acc(gold_index,qa_index)
print(acc)


- prepare relevance training data
assign label 1 denote relevance, label0 denote irrelevance
randomly choose incorrect query-answer pairs in trainset and labeled 0
choose top N correct query-answer pairs in trainset and labeled 1

In [464]:
model.wv.most_similar(['movie'], topn=10)

[('film', 0.6140520572662354),
 ('spielberg', 0.43539032340049744),
 ('console', 0.4009501338005066),
 ('sylvester', 0.3827870786190033),
 ('animate', 0.3810267746448517),
 ('tv', 0.37869757413864136),
 ('entertainment', 0.37179452180862427),
 ('stallone', 0.36797311902046204),
 ('tudio', 0.3666054904460907),
 ('television', 0.3636029362678528)]

def prepare_relevance_data(N):
    file = json.load(open(train_set))
    query = [x['question'] for x in file]
    answer = [x['text'] for x in file]

    pair = []
    for i in range(N):
        q = query[i]
        a = answer[i]
        pair.append(((q,a),1))
        random_q = random_a = [i]
    
        while random_q[0] == i:
            random_q = np.random.choice(N,1)
        while random_a[0] == i or random_a[0] == random_q[0]:
            random_a = np.random.choice(N,1)
        q = query[random_q[0]]  
        a = answer[random_a[0]]
        pair.append(((q,a),0))
    return pair

trainset = prepare_relevance_data(40000)
devset = prepare_relevance_data(3097)
    


    
    

In [465]:
def transform_query(query,answer):
    q = [x.lower().strip('?') for x in query.split()]
    trans_form_query = []
    for word in q:
        if word == 'what' or \
        word == 'who' or \
        word == 'where' or \
        word == 'when' or \
        word == 'which'or \
        word == 'how'or \
        word == 'whose':
            word = str(answer)
        trans_form_query.append(word)
    q = ' '.join(trans_form_query)
    return q    

#just a demo, need more pre-extracted potential answer sets

import ast
def prepare_train(N,dataset):
    file = json.load(open(dataset))
    f = open(log_file,"r")
    corrects = []
    wrongs = []
    for i,tops in enumerate(f):
        line = file[i]
        query = line['question']
        answer = line['text']
        correct = transform_query(query,answer)
        corrects.append(correct)
        topk = ast.literal_eval(tops.strip())
        constituents = get_para_constituent(topk)
        for n in range(5):
            cons = np.random.choice(constituents,1)
            wrong = transform_query(query, cons[0])
            wrongs.append(wrong)
    return corrects, wrongs

correct, wrong = prepare_train(4155,train_set)
label = []

for i in correct:
    label.append(1)
for i in wrong:
    label.append(0)
    
print("done")
            
    

print(len(correct))
print(len(wrong))

def prepare_feature_vector(correct,wrong):
    matrix = []
    for i in correct:
        vector = sent2vec(i, model, 160, index2word_set,'disable')
        matrix.append(vector)
    for i in wrong:
        vector = sent2vec(i, model, 160, index2word_set,'disable')
        matrix.append(vector)
    return matrix

X = prepare_feature_vector(correct,wrong)
print("done")


print(len(X))
print(len(label))

- binary task, long feature vector -> take SVM classifier

def train(X,y):   
    model = LogisticRegression()
    best_model = model.fit(X, y)
    return best_model
            
relevance_model = train(X,label)
print("done")

def pred(query,entity,classifier):
    
    q = transform_query(query,entity)
    vector = sent2vec(q, model, 160, index2word_set,'disable')
    preds = classifier.predict([vector][0:])
    return preds
    
pred = pred("What example is given as another paired relationship of uncertainly related to standard deviation?",
            "time vs. energy",
           relevance_model)

q = []
l = []
for i in range(4741):
    q.append(sent2vec(wrong[i], model, 160, index2word_set,'disable'))
    l.append(0)
for i in correct:
    q.append(sent2vec(i, model, 160, index2word_set,'disable'))
    l.append(1)
    
preds = relevance_model.predict(q[0:])
    
    

# from sklearn.externals import joblib

#save clf
joblib.dump(relevance_model, '/Users/alfredchen/Documents/GitHub/query-system/models/relevance.pkl')

#load clf
#relevance_model = joblib.load('/Users/alfredchen/Documents/GitHub/query-system/models/relevance.pkl') 

def relevant(query, answer, classifier):
    X = []
    for i in range(len(query)):
        q = transform_query(query[i],answer[i])
          
        x = sent2vec(q, model, 160, index2word_set,'disable')
        #x = np.concatenate((q,a),axis=0)
        X.append(x)
    preds=relevance_model.predict(X[0:])
    return preds

#dev set

query=[]
answer = []
labels = []
for pair,label in devset:
    query.append(pair[0])
    answer.append(pair[1])
    labels.append(label)
    
preds = relevant(query,answer,relevance_model)
    

def print_metrices(y_test,y_pred):
    print("Accuracy:")
    print(metrics.accuracy_score(y_test,y_pred))

    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred,average='macro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred,average='macro'))
    
    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred,average='macro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred))
    
print_metrices(l, preds)

def keywords_extraction(query):
    query = query.split(' ')
    query = [removePunc(x) for x in query]
    query = [lemmatize(x.lower()) for x in query]
    q = []
    for w in query:
        if not stopwords.get(w) and w != '':
            q.append(w)
    print(q)
    tagged = nltk.pos_tag(q,tagset="universal")
    return [p[0] for p in tagged if p[1] in ["ADJ","NOUN","VERB","NUM"]]


def relation_extraction(query):
    words = keywords_extraction(query)
    if words == []:
        return
    relations = defaultdict(dict)
    for w1 in words:
        for w2 in words:
            if w1 == w2:
                continue
            else:
                relations[w1][w2] = relations[w1].get(w2, 0) + 1
    return relations

pos = keywords_extraction("a kilogram can be defined as what value")
print(pos)
relations = relation_extraction("a kilogram can be defined as what value?")
print(relations)




def answer_type(query):
    query = query.split()
    query = [removePunc(x) for x in query]
    query = [lemmatize(x.lower()) for x in query]
    for word in query:
        if word == 'what':
            return 'O'
        elif word == 'who':
            return 'PERSON'
        elif word == 'where':
            return 'LOCATION'
        elif word == 'when':
            return 'DATE'
        elif word == 'which':
            return 'O'
    return 'O'
        

# three round to extract a list of potential answers from topk paragraphs

    - PERSON                -People, including fictional.
    - NORP	              -Nationalities or religious or political groups.
    - FACILITY	          -Buildings, airports, highways, bridges, etc.
    - ORG	                  -Companies, agencies, institutions, etc.
    - GPE	                  -Countries, cities, states.
    - LOC	                  -Non-GPE locations, mountain ranges, bodies of water.
    - PRODUCT	              -Objects, vehicles, foods, etc. (Not services.)
    - EVENT	              -Named hurricanes, battles, wars, sports events, etc.
    - WORK_OF_ART	          -Titles of books, songs, etc.
    - LAW	Named             -documents made into laws.
    - LANGUAGE	          -Any named language.
    - DATE	              -Absolute or relative dates or periods.
    - TIME	              -Times smaller than a day.
    - PERCENT	              -Percentage, including "%".
    - MONEY	              -Monetary values, including unit.
    - QUANTITY	          -Measurements, as of weight or distance.
    - ORDINAL	              -"first", "second", etc.
    - CARDINAL	          -Numerals that do not fall under another type.


In [466]:
# set up classifier for detect answer type
def detect_answer_type(query):
    return answer_type

#preprocess word in query
def preprocess_sentence(sentence):
    words = sentence.split(' ')
    words = [lemmatize(removePunc(x.lower())) for x in words]
    return ' '.join(words)

def named_entity_recognize(sentence,valid_ner):
    #do not preprocess sentence when getting named entity using spacy
    """
    """
    entities = []
    ner_tag = []
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sentence)
    if valid_ner == 'disable':
        ner_tag = ''
        for ent in doc.ents:
            ner_tag=ent.label_
        if ner_tag == '':
            ner_tag = 'O'
        return ner_tag
    for ent in doc.ents:
        if ent.label_ in valid_ner:
            entities.append(ent)
            ner_tag.append(ent.label_)
    # case 'O'
    if valid_ner == 'O':
        sentence = preprocess_sentence(sentence)
        ents = sentence.split(' ')
        
        """
        add more selection method
        """
        entities += ents
        ner_tag.append('O')
        if entities == []:
            return 'unknown'
    return entities

entities = named_entity_recognize("January 11, 2017",['DATE','TIME','CARDINAL','GPE'])  

    




In [467]:
# apply ner recognition to get all answer type from training data
"""
def tag_answer(dataset):
    file = json.load(open(dataset))
    for _id,line in enumerate(file):
        answer = line['text']
        label = named_entity_recognize(answer,'disable')
        f = open(answer_type,"a")
        f.write(str(_id)+':'+str(label)+'\n')
        f.close()
    print('done')
    return True

tag_answer(train_set)
""" 
pass
        



In [468]:
# train a classifier to recognize answer type from query using top 18173 data)

def rule(sentence):
    sentence = preprocess_sentence(sentence).split(' ')
    rule = {
        'person':'PERSON',
        'location': 'GPE',            
        'who': 'PERSON',
        'why': 'O',
        'from': 'GPE',
        'country': 'GPE',
        'capital': 'GPE',
        'city': 'GPE',
        'when': 'DATE',
        'many': 'CARDINAL',
        'long':'CARDINAL',
        'high': 'CARDINAL',
        'year': 'DATE',
        'decade': 'CARDINAL',
        'time': 'DATE',
        'cost': 'MONEY',
        'population': 'CARDINAL',
        'number':'CARDINAL'
    }
    for word in sentence:
        if rule.get(word):
            return rule.get(word)
    return 'O'

def query2vector(dataset):
    
    label =  open(answer_type_file)
    train = json.load(open(dataset))
    X = []
    Y = []
    for i,line in enumerate(label):
        x = sent2vec(train[i]['question'], model, 160, index2word_set, 'disable')
        y = line.split(':')[1].strip()
        if y == 'O':
            y=rule(train[i]['question'])
        X.append(x)
        Y.append(y)
    label.close()
    return X,Y
    
vectors,labels = query2vector(train_set)
print(len(vectors))
print(len(labels))
        
        
    






18174
18174


In [290]:
# logistic classifier
def train(X,y):   
    clf = LogisticRegression()
    best_model = clf.fit(X, y)
    return best_model
            
answer_type = train(vectors,labels)
print("done")

done


In [291]:
def pred_answer_type(query,classifier):
    q = sent2vec(query, model, 160, index2word_set, 'disable')
    preds = classifier.predict([q][0:])
    return preds[0]

print(pred('how much do you spend',answer_type))

MONEY


assumptions

        1. answers are not in the query and are not stopwords
        2. answers are semantically close to query keywords
        

In [453]:
#

import math

def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    """https://en.wikipedia.org/wiki/Cosine_similarity"""
    num = np.dot(v1, v2)
    d1 = np.dot(v1, v1)
    d2 = np.dot(v2, v2)
    if d1 > 0.0 and d2 > 0.0:
        return num / math.sqrt(d1 * d2)
    else:
        return 0.0



stop_words = get_stopwords(False) 
def first_filter(query,answer):
    """
    remove stopwords in answers
    remove duplicate answers
    remove words in query
    """
    query = preprocess_sentence(query).split(' ')
    q = []
    a = []
    template = []
    for w in query:
        if not stop_words.get(w):
            q.append(w)
    for w in answer:
        try:
            flag = 1
            words = str(w.text).split(' ')
            words = [lemmatize(removePunc(x).lower()) for x in words]
            for word in words:
                if not stop_words.get(word):
                    if word not in template:
                        if word in q:
                            flag = 0
                    else:
                        flag = 0
                else:
                    flag = 0
            if flag == 1:
                a.append(w.text)
                template += words
        except AttributeError:
            word = lemmatize(w.lower())
            if not stop_words.get(word):
                if word not in template:
                    if word not in q:
                        a.append(w)
                        template.append(word)
    q = ' '.join(q)
    return q,a

def second_filter(query,answer):
    """
    the most potential answer is semantically most similar to query
    """
    query, answer = first_filter(query,answer)
    query_semantic = sent2vec(query, model, 160, index2word_set, 'disable')
    max_sim = 1
    potential_answer = ''
    for a in answer:
        tmp = lemmatize(a.lower())
        ans = sent2vec(a, model, 160, index2word_set, 'disable')
        sim = cosine_similarity(query_semantic,ans)
        if sim > 0:
            return a
            print(a)
            max_sim = sim
            potential_answer = a
    return potential_answer
            
        
    

            
        
    

In [454]:
def extract_answer_from_topk(query,topk):
    context = ""
    for index in topk:
        context += index2para.get(tuple(index))
    valid_ner = pred_answer_type(query,answer_type)
    if valid_ner == 'O':
        valid_ner = rule(query)
    potential_answers = named_entity_recognize(context,valid_ner)
    answer = second_filter(query,potential_answers)
    return answer

topk = getTopN(10,"what is the capital of England?",'disable')
query = "what is the capital of England?"

answers = extract_answer_from_topk(query,topk)
print(answers)



London


In [475]:
# test acc on devset
def test_answers(dataset):
    file = json.load(open(dataset))
    #top_k = open(topk_file)
    gold = []
    pred = []
    for _id,line in enumerate(file):
        query = line['question']
        topk = getTopN(10,query,line['docid'])
        answer = extract_answer_from_topk(query,topk)
        gold.append(line['text'])
        pred.append(answer)
        if _id % 50 == 0:
            print(metrics.accuracy_score(gold,pred))
            
    return gold,pred

gold,pred = test_answers(dev_set)
        

0.0
0.0392156862745098
0.04950495049504951
0.039735099337748346
0.029850746268656716
0.043824701195219126
0.04318936877076412
0.039886039886039885
0.0399002493765586
0.037694013303769404
0.03992015968063872
0.039927404718693285
0.036605657237936774
0.03533026113671275
0.03281027104136947
0.033288948069241014
0.031210986267166042
0.03407755581668625
0.03440621531631521
0.03680336487907466
0.03896103896103896
0.037107516650808754
0.03814713896457766
0.03735881841876629
0.03746877601998335
0.037569944044764186
0.03766333589546503
0.03626943005181347
0.03568879371877231
0.03583735354927636
0.03530979347101932
0.03481624758220503
0.03560274828232355
0.03513022410660206
0.03409758965314521
0.03312392918332382
0.03220433092726263
0.031334413830361965
0.03103629668595476
0.03177857508969759
0.031984007996002
0.031204290589956118
0.03236554021894336
0.03161320316132032
0.03225806451612903
0.03198578409595735
0.03129074315514994
0.03317737133134836
0.03373594335693461
0.03345573235414117
0.03318

In [ ]:
def print_metrices(y_test,y_pred): 
    print("Accuracy:") 
    print(metrics.accuracy_score(y_test,y_pred))
    
    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred,average='macro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred,average='macro'))

    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred,average='macro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred))

print_metrices(gold, pred)